<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/NIS_march.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **НИС SENTI-FRAME**

In [5]:
!pip install pymorphy2[fast]
#!rm collection.json.1 collection.json
#!ls
import gensim
from collections import Counter
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


## **Импорт с Rusvectores skipgram и fasttext модлей, обученных на корпусе Тайга.**

In [2]:
!wget https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json
!wget rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz
!wget http://vectors.nlpl.eu/repository/20/187.zip
!unzip 187.zip

--2020-03-10 08:11:53--  https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 606414 (592K) [text/plain]
Saving to: ‘collection.json’

collection.json     100%[===================>] 592.20K  --.-KB/s    in 0.04s   

2020-03-10 08:11:54 (15.4 MB/s) - ‘collection.json’ saved [606414/606414]

--2020-03-10 08:11:55--  http://rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549952184 (524M) [application/x-gzip]
Saving to: ‘news_mystem_skipgram_1000_20_2015.bin.gz’


In [6]:
collection_handler = open("collection.json", "r", encoding="utf-8")
collection = json.load(collection_handler, encoding="utf-8")
skipgram_model = gensim.models.KeyedVectors.load_word2vec_format('news_mystem_skipgram_1000_20_2015.bin.gz', binary=True)
fasttext_model = gensim.models.KeyedVectors.load("model.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## **В процессе формирования сидовых множеств выяснилось, что косинусная дистанция очень сильно зависит от количества слов в сидовом множестве. Вот несколько примеров:**

In [0]:
seed_1_dict = ['одобрять']
seed_2_dict = ['негодовать']
seed_3_dict = ['убивать']
seed_4_dict = ['хвалить']
seed_5_dict = ['убивать', 'негодовать']
seed_6_dict = ['хвалить', 'одобрять']
def make_seed_embedding(seed_dict, model):
  '''
  Принимает на вход список, на выходе дает средний вектор всех слов из этого списка
  '''
  summ = 0
  if model == skipgram_model:
    for verb in seed_dict:
      summ = summ + model[verb + '_V']
    vector = summ/len(seed_dict)
  else:
    for verb in seed_dict:
      summ = summ + model.get_vector(verb)
    vector = summ/len(seed_dict)

  return vector
seed_1_skipgram = make_seed_embedding(seed_1_dict, skipgram_model)
seed_1_fasttext = make_seed_embedding(seed_1_dict, fasttext_model)

seed_2_skipgram = make_seed_embedding(seed_2_dict, skipgram_model)
seed_2_fasttext = make_seed_embedding(seed_2_dict, fasttext_model)

seed_3_skipgram = make_seed_embedding(seed_3_dict, skipgram_model)
seed_3_fasttext = make_seed_embedding(seed_3_dict, fasttext_model)

seed_4_skipgram = make_seed_embedding(seed_4_dict, skipgram_model)
seed_4_fasttext = make_seed_embedding(seed_4_dict, fasttext_model)

seed_5_skipgram = make_seed_embedding(seed_5_dict, skipgram_model)
seed_5_fasttext = make_seed_embedding(seed_5_dict, fasttext_model)


seed_6_skipgram = make_seed_embedding(seed_6_dict, skipgram_model)
seed_6_fasttext = make_seed_embedding(seed_6_dict, fasttext_model)


In [34]:
similarity = cosine_distances(np.atleast_2d(seed_1_skipgram), np.atleast_2d(seed_2_skipgram))
similarity2 = cosine_distances(np.atleast_2d(seed_3_skipgram), np.atleast_2d(seed_4_skipgram))
similarity3 = cosine_distances(np.atleast_2d(seed_5_skipgram), np.atleast_2d(seed_6_skipgram))
print(similarity, 'одобрять-негодовать')
print(similarity2, 'убивать-хвалить')
print(similarity3, 'одобрять, негодовать - убивать, хвалить')

[[0.8885118]] одобрять-негодовать
[[0.8539348]] убивать-хвалить
[[0.6935718]] одобрять, негодовать - убивать, хвалить


In [0]:
def add_sim_list(model, my_verbs_list, seed_vector):
  '''
  Принимает на вход модель, как назвать результирующую колонку в датафрейме,
  словарь из глаголов, и средний вектор сидового множества.
  записывает колонку в датафрейм, возвращает словарь косинусных расстояний
  '''
  sim_list = []
  if model == skipgram_model:  
    for verb in my_verbs_list:
        try:
          similarity = cosine_distances(np.atleast_2d(model[verb + '_V']), np.atleast_2d(seed_vector))
        except KeyError as e:
          similarity = 'verb is not in model dict'
        sim_list.append(similarity)
  else:
    for verb in my_verbs_list:
      try:
        similarity = cosine_distances(np.atleast_2d(model.get_vector(verb)), np.atleast_2d(seed_vector))
      except (KeyError, AttributeError) as e:
        similarity = 'verb is not in model dict'
      sim_list.append(similarity)
  print(len(sim_list))
  print(len(my_verbs_list))

  #dataframe[df_pos] = sim_list
  return sim_list